os - file & folder handling

pandas - reading/writing CSVs

numpy - numerical operations

torch - deep learning (LSTM)

timedelta - generating future dates

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datetime import timedelta

In [3]:
SEQ_LEN = 14 #use 14 days data to predict for next 7 days
PRED_DAYS = 7
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

POLLUTANTS = ["co", "pm25", "pm10", "no2", "so2", "o3"]

In [4]:
PROJECT_ROOT = os.path.abspath("..")          # AQI/
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
PREDICTIONS_DIR = os.path.join(DATA_DIR, "predictions")
MODELS_DIR = os.path.join(PROJECT_ROOT, "POLLUTANT_FORECASTING", "models")

os.makedirs(PREDICTIONS_DIR, exist_ok=True)

print("Processed data path:", PROCESSED_DIR)
print("Models path:", MODELS_DIR)
print("Predictions will be saved to:", PREDICTIONS_DIR)

Processed data path: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\processed
Models path: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\POLLUTANT_FORECASTING\models
Predictions will be saved to: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\predictions


In [5]:
 class LSTMPredictor(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

Input: 1 value per timestep (pollutant value)

LSTM layers: 2 layers, 64 hidden units

Fully Connected layer: converts last LSTM output → prediction

In [6]:
def predict_future(model, last_seq, days):
    model.eval()
    seq = last_seq.copy()
    preds = []

    for _ in range(days):
        x = torch.tensor(seq[-SEQ_LEN:], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            pred = model(x).item()
        preds.append(pred)
        seq = np.append(seq, [[pred]], axis=0)

    return np.array(preds)

1.Take last 14 normalized values

2.Predict day-1

3.Append prediction to sequence

4.Use updated sequence to predict day-2

5.Repeat until 7 days

In [7]:
for pollutant in POLLUTANTS:
    print(f"\n Predicting {pollutant.upper()}")

    csv_path = os.path.join(PROCESSED_DIR, f"{pollutant}_clean.csv")
    model_path = os.path.join(MODELS_DIR, f"lstm_{pollutant}.pt")

    if not os.path.exists(csv_path):
        print(f" Missing CSV: {csv_path}")
        continue

    if not os.path.exists(model_path):
        print(f" Missing model: {model_path}")
        continue

    # Load data
    df = pd.read_csv(csv_path, parse_dates=["date"])
    df = df.sort_values("date")

    series_norm = df["value_norm"].values.reshape(-1, 1)
    last_seq = series_norm[-SEQ_LEN:]

    # Load model
    model = LSTMPredictor().to(DEVICE)
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))

    # Predict
    preds_norm = predict_future(model, last_seq, PRED_DAYS)

    # Un-normalize
    min_val = df["value"].min()
    max_val = df["value"].max()
    preds = preds_norm * (max_val - min_val) + min_val

    # Dates
    last_date = df["date"].max()
    future_dates = [last_date + timedelta(days=i) for i in range(1, PRED_DAYS + 1)]

    # Save CSV
    out_df = pd.DataFrame({
        "date": future_dates,
        f"{pollutant}_pred": preds
    })

    out_path = os.path.join(PREDICTIONS_DIR, f"{pollutant}_predictions.csv")
    out_df.to_csv(out_path, index=False)

    print(f" Saved: {out_path}")

print("\n ALL POLLUTANT PREDICTIONS SAVED SUCCESSFULLY!")



 Predicting CO
 Saved: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\predictions\co_predictions.csv

 Predicting PM25
 Saved: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\predictions\pm25_predictions.csv

 Predicting PM10
 Saved: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\predictions\pm10_predictions.csv

 Predicting NO2
 Saved: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\predictions\no2_predictions.csv

 Predicting SO2
 Saved: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\predictions\so2_predictions.csv

 Predicting O3
 Saved: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\predictions\o3_predictions.csv

 ALL POLLUTANT PREDICTIONS SAVED SUCCESSFULLY!


In [4]:
# import os
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from datetime import timedelta

# # -------------------------------
# # CONFIG
# # -------------------------------
# SEQ_LEN = 14
# PRED_DAYS = 7
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# # -------------------------------
# # PATHS
# # -------------------------------
# PROJECT_ROOT = "CDAC_DBDA_PROJECT/AQI"  # root of your AQI project
# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
# MODELS_DIR = os.path.join(PROJECT_ROOT, "POLLUTANT_FORECASTING", "models")
# PREDICTIONS_DIR = os.path.join(DATA_DIR, "predictions")
# os.makedirs(PREDICTIONS_DIR, exist_ok=True)

# POLLUTANTS = ["co", "pm25", "pm10", "no2", "so2", "o3"]

# # -------------------------------
# # LSTM MODEL CLASS
# # -------------------------------
# class LSTMPredictor(nn.Module):
#     def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=1):
#         super().__init__()
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
    
#     def forward(self, x):
#         out, _ = self.lstm(x)
#         out = self.fc(out[:, -1, :])
#         return out

# # -------------------------------
# # FUTURE PREDICTION FUNCTION
# # -------------------------------
# def predict_future(model, last_seq, days=PRED_DAYS):
#     model.eval()
#     seq = last_seq.copy()
#     preds = []
#     for _ in range(days):
#         x = torch.tensor(seq[-SEQ_LEN:], dtype=torch.float32).unsqueeze(0).to(DEVICE)
#         with torch.no_grad():
#             pred = model(x).cpu().numpy()[0][0]
#         preds.append(pred)
#         seq = np.append(seq, [[pred]], axis=0)
#     return np.array(preds)

# # -------------------------------
# # MAIN LOOP
# # -------------------------------
# for pollutant in POLLUTANTS:
#     csv_file = os.path.join(PROCESSED_DIR, f"{pollutant}_clean.csv")
#     model_file = os.path.join(MODELS_DIR, f"lstm_{pollutant}.pt")

#     if not os.path.exists(csv_file):
#         print(f"Skipping {pollutant.upper()}: CSV not found at {csv_file}")
#         continue
#     if not os.path.exists(model_file):
#         print(f"Skipping {pollutant.upper()}: Model not found at {model_file}")
#         continue

#     # Load processed CSV
#     df = pd.read_csv(csv_file, parse_dates=['date'])
#     df = df.sort_values('date')

#     if "value_norm" not in df.columns or "value" not in df.columns:
#         print(f"Skipping {pollutant.upper()}: required columns missing")
#         continue

#     series_norm = df["value_norm"].values.reshape(-1, 1)
#     last_seq = series_norm[-SEQ_LEN:]

#     # Load model
#     model = LSTMPredictor(input_dim=1).to(DEVICE)
#     model.load_state_dict(torch.load(model_file, map_location=DEVICE))

#     # Predict next 7 days
#     future_norm = predict_future(model, last_seq, days=PRED_DAYS)

#     # Un-normalize predictions
#     min_val = df["value"].min()
#     max_val = df["value"].max()
#     future_values = future_norm * (max_val - min_val) + min_val

#     # Generate future dates
#     last_date = df['date'].max()
#     future_dates = [last_date + timedelta(days=i) for i in range(1, PRED_DAYS + 1)]

#     # Save predictions to CSV in predictions folder
#     pred_df = pd.DataFrame({
#         "date": future_dates,
#         f"{pollutant}_pred": future_values
#     })
#     pred_file = os.path.join(PREDICTIONS_DIR, f"{pollutant}_predictions.csv")
#     pred_df.to_csv(pred_file, index=False)
#     print(f"{pollutant.upper()} predictions saved to {pred_file}")

# print("\nAll predictions saved successfully in 'data/predictions/' folder.")


Skipping CO: CSV not found at CDAC_DBDA_PROJECT/AQI\data\processed\co_clean.csv
Skipping PM25: CSV not found at CDAC_DBDA_PROJECT/AQI\data\processed\pm25_clean.csv
Skipping PM10: CSV not found at CDAC_DBDA_PROJECT/AQI\data\processed\pm10_clean.csv
Skipping NO2: CSV not found at CDAC_DBDA_PROJECT/AQI\data\processed\no2_clean.csv
Skipping SO2: CSV not found at CDAC_DBDA_PROJECT/AQI\data\processed\so2_clean.csv
Skipping O3: CSV not found at CDAC_DBDA_PROJECT/AQI\data\processed\o3_clean.csv

All predictions saved successfully in 'data/predictions/' folder.
